<a href="https://colab.research.google.com/github/simonebugo/Big_Data/blob/main/Esercitazione_esame_1(pipeline).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Il dataset (preso da kaggle https://www.kaggle.com/datasets/iabhishekofficial/mobile-price-classification) contiene dati relativi ad alcuni
cellulari. Il dataset contiene diverse feature descritte di seguito, si vuole predire il valore di range di prezzo sulla base degli attributi presenti
nel dataset:

● battery_power: potenza della batteria (mAh)

● blue: ha il bluetooth oppure no (boolean)

● clock_speed: velocità del microprocessore

● dual_sim: ha il supporto dual sim oppure no (boolean)

● fc: mega pixels della telecamera frontale

● four_g: ha il 4G oppure no (boolean)

● int_memory: nemoria interna (GB)

● m_dep: Mobile Depth (cm)

● mobile_wt: peso

● n_cores: numero di core del processore

● pc: mega pixels della fotocamera esterna

● px_height: Pixel Resolution Height

● px_width: Pixel Resolution Width

● ram: RAM (MB)

● sc_h: altezza schermo (cm)

● sc_w: larghezza schermo (cm)

● talk_time: durata massima della batteria

● three_g: ha il 3G oppure no (boolean)

● touch_screen: ha il touch screen oppure no (boolean)

● wifi: ha il wifi oppure no (boolean)

● price_range: colonna target con valori 0 (costo basso), 1 (costo medio), 2 (costo alto), 3 (costo molto alto)

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("dataset.csv")
df.head() #target è price_range

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [24]:
df["price_range"].value_counts() #verifico se è bilanciato

,count
price_range,
1,500
2,500
3,500
0,500


In [7]:
df.dtypes

,0
battery_power,int64
blue,int64
clock_speed,float64
dual_sim,int64
fc,int64
four_g,int64
int_memory,int64
m_dep,float64
mobile_wt,int64
n_cores,int64


5. Creare una pipeline in cui gli attributi int_memory, ram e talk_time sono scalati in
modo che abbiano media 0 e varianza 1, gli attributi mobile_wt e battery_power sono
discretizzati in 5 intervalli, e tutti gli altri attributi sono lasciati invariati. La pipeline deve
applicare il modello DecisionTree. Valutare l’accuratezza della classificazione attraverso
accuracy e confusion matrix.

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import KBinsDiscretizer, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

In [28]:
#function to test the model and to validate it with the function requested (accuracy and confusion matrix)
def test_model(name, model, traindata, testdata, trainlabels, testlabels):
  model.fit(traindata, trainlabels)
  trainpred = model.predict(traindata)
  testpred = model.predict(testdata)
  print(f" ===== {name} =====")
  print(f"accuracy on train {accuracy_score(trainlabels,trainpred)}")
  print(f"confusion matrix on train \n {confusion_matrix(trainlabels,trainpred)}")
  print(f"accuracy on test {accuracy_score(testlabels,testpred)}")
  print(f"confusion matrix on test \n {confusion_matrix(testlabels,testpred)}")


In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(df.drop('price_range', axis=1), df['price_range'], test_size=0.25,stratify = df['price_range'], random_state=123)
col_tran = ColumnTransformer(
    transformers=[
        ('std', StandardScaler(),["int_memory","ram","talk_time"]),
        ('discr', KBinsDiscretizer(n_bins = 5), ["mobile_wt","battery_power"])
        ],
    remainder='passthrough'#per lasciare invariati gli altri attributi
)
pipeline = Pipeline(steps=[
                              ('col_tran', col_tran),
                              ('model', DecisionTreeClassifier(random_state = 123)),
                             ])
test_model("Pipeline",pipeline,X_train,X_test,Y_train,Y_test)

 ===== Pipeline =====
accuracy on train 1.0
confusion matrix on train 
 [[375   0   0   0]
 [  0 375   0   0]
 [  0   0 375   0]
 [  0   0   0 375]]
accuracy on test 0.788
confusion matrix on test 
 [[108  17   0   0]
 [ 15  93  17   0]
 [  0  22  88  15]
 [  0   0  20 105]]


6. Aggiungere alla pipeline del punto 5 la funzione SelectKBest per selezionare K feature tra
quelle restituite dalla pipeline del punto 5. Utilizzare la funzione di gridSearchCV per
selezionare il valore migliore di K, il numero migliore di bin in cui discretizzare i valori di
mobile_wt e battery_power e i valori degli iperparametri criterion e
min_samples_leaf del modello DecisionTree (scegliere a piacere alcuni valori).

In [21]:
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV

In [39]:
# Definiamo la nuova Pipeline con SelectKBest
pipeline_kbest = Pipeline(steps=[
    ('coltran', col_tran),
    ('kbest', SelectKBest()),
    ('estimator', DecisionTreeClassifier(random_state=123))
])

pipeline_kbest

Pipeline(steps=[('coltran',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('std', StandardScaler(),
                                                  ['int_memory', 'ram',
                                                   'talk_time']),
                                                 ('discr', KBinsDiscretizer(),
                                                  ['mobile_wt',
                                                   'battery_power'])])),
                ('kbest', SelectKBest()),
                ('estimator', DecisionTreeClassifier(random_state=123))])

In [41]:
parameters = {
    'coltran__discr__n_bins' : [20,30,50],
    'kbest__k' : [5,"all"],
    'estimator__criterion' : ["gini","entropy"],
    'estimator__min_samples_leaf' : [4,5,6,9,10]
}

gd = GridSearchCV(pipeline_kbest, parameters)
gd.fit(X_train,Y_train)
Y_pred = gd.predict(X_test)
print(f"bestparam {gd.best_params_}")
print(f"bestscore {gd.best_score_}") #se in esame ti chiede se le performance migliorano o peggiornao scrivi una frase

bestparam {'coltran__discr__n_bins': 20, 'estimator__criterion': 'gini', 'estimator__min_samples_leaf': 5, 'kbest__k': 'all'}
bestscore 0.768


7. Creare una nuova pipeline che applica la decomposizione TruncatedSVD al dataset
iniziale e aggiunge le componenti ottenute alla pipeline del punto 5. Valutare il valore
migliore per il numero di componenti di TruncatedSVD tra 2, 4 e 6.

In [42]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import FeatureUnion

In [44]:
combined_features = FeatureUnion([("svd",TruncatedSVD()),("Coltran",col_tran)])
my_pipeline = Pipeline(steps=[
    ('combined_features', combined_features),
    ('estimator',DecisionTreeClassifier(random_state=123))
                              ],
                       verbose = True)
my_pipeline

Pipeline(steps=[('combined_features',
                 FeatureUnion(transformer_list=[('svd', TruncatedSVD()),
                                                ('Coltran',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   transformers=[('std',
                                                                                  StandardScaler(),
                                                                                  ['int_memory',
                                                                                   'ram',
                                                                                   'talk_time']),
                                                                                 ('discr',
                                                                                  KBinsDiscretizer(),
                                                                                  ['mobile_wt',
                                                                                   'battery_power'])]))])),
                ('estimator', DecisionTreeClassifier(random_state=123))],
         verbose=True)

In [46]:
parameters = {
    'combined_features__svd__n_components' : [2,4,6]
}
gd = GridSearchCV(my_pipeline, parameters)
gd.fit(X_train,Y_train)
Y_pred = gd.predict(X_test)
print(f"Accuracy {accuracy_score(Y_test,Y_pred)}")
print(f"confusion matrix \n {confusion_matrix(Y_test,Y_pred)}")
print(f"bestparam {gd.best_params_}")

[Pipeline] . (step 1 of 2) Processing combined_features, total=   0.1s
[Pipeline] ......... (step 2 of 2) Processing estimator, total=   0.0s
[Pipeline] . (step 1 of 2) Processing combined_features, total=   0.1s
[Pipeline] ......... (step 2 of 2) Processing estimator, total=   0.0s
[Pipeline] . (step 1 of 2) Processing combined_features, total=   0.0s
[Pipeline] ......... (step 2 of 2) Processing estimator, total=   0.0s
[Pipeline] . (step 1 of 2) Processing combined_features, total=   0.1s
[Pipeline] ......... (step 2 of 2) Processing estimator, total=   0.0s
[Pipeline] . (step 1 of 2) Processing combined_features, total=   0.0s
[Pipeline] ......... (step 2 of 2) Processing estimator, total=   0.0s
[Pipeline] . (step 1 of 2) Processing combined_features, total=   0.0s
[Pipeline] ......... (step 2 of 2) Processing estimator, total=   0.0s
[Pipeline] . (step 1 of 2) Processing combined_features, total=   0.1s
[Pipeline] ......... (step 2 of 2) Processing estimator, total=   0.0s
[Pipel